In [1]:
%pip install numpy
%pip install pandas
%pip install sklearn
import pandas as pd
import numpy as np
import os, psutil
import warnings
import time
warnings.filterwarnings('ignore')

from sklearn.ensemble import (StackingClassifier, VotingClassifier, RandomForestClassifier)
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, classification_report, matthews_corrcoef, roc_auc_score, cohen_kappa_score, precision_score, mean_absolute_error, mean_squared_error 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 64.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 92.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 58.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 48.8 MB/s eta 0:00:0000:0100:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=f957c0af3aafde9c3b5c82a69711b8c019cf111318711644a19d3ac3a50ff5dc
  Stored in directory: /home/vikas/.cache/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [2]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment',
            'urgent','hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted',
            'num_root','num_file_creations','num_shells','num_access_files','num_outbound_cmds',
            'is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate',
            'srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count',
            'dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
            'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
            'dst_host_srv_rerror_rate','attack','level'])

In [3]:
def cpu_stats():
    pid = os.getpid()
    py = psutil.Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return 'memory GB:' + str(np.round(memory_use, 2))

def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

print('Function built')

Function built


In [4]:
df_train = pd.read_csv("/project/KDDTrain+.txt")
df_train.columns = columns
df_test = pd.read_csv("/project/KDDTest+.txt")
df_test.columns = columns
df_train = reduce_memory_usage(df_train, verbose=True)
df_test = reduce_memory_usage(df_test, verbose=True)
print(cpu_stats())
print('Memory reduced')

Mem. usage decreased to 12.13 Mb (70.6% reduction)
Mem. usage decreased to 2.17 Mb (70.6% reduction)
memory GB:0.23
Memory reduced


In [5]:
le = LabelEncoder()
df_train['protocol_type'] = le.fit_transform(df_train['protocol_type'])
df_test['protocol_type'] = le.transform(df_test['protocol_type'])
df_train['service'] = le.fit_transform(df_train['service'])
df_test['service'] =le.transform(df_test['service'])
df_train['flag'] = le.fit_transform(df_train['flag'])
df_test['flag'] = le.transform(df_test['flag'])

In [6]:
label = []
for i in df_train.attack :
  if i == 'normal':
    label.append(0)
  else:
    label.append(1)
df_train['label'] = label 

label_test = []
for i in df_test.attack :
  if i == 'normal':
    label_test.append(0)
  else:
    label_test.append(1)
df_test['label'] = label_test

In [7]:
df_train.drop('attack',axis = 1,inplace = True)
df_test.drop('attack',axis = 1,inplace = True)
y_test = df_test['label'].ravel()
df_test.drop('label', axis=1, inplace=True)
y_train = df_train['label'].ravel()
train = df_train.drop(['label'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = df_test.values # Creats an array of the test data

results = pd.DataFrame(columns = ['S.no', 'Model', 'Execution Time', 'Accuracy', 'F1-score', 'Precision', 
                                  'Recall', 'MCC', 'ROC', 'Kappa', 'Mean Absolute Error', 'Root Mean Squared Error', 'Relative Absolute Error', 'Root Relative Squared Error'])
num = 0

In [8]:
import math
def get_report(actual, pred, model, t):
    global num, results
    print("Confusion matrix: \n", confusion_matrix(actual, pred))
    accuracy = accuracy_score(actual, pred)
    print("Accuracy: ", accuracy)
    fscore = f1_score(actual, pred)
    print("F1 Score: ", fscore)
    precision = precision_score(actual, pred)
    print("Precision Score: ", precision)
    recall = recall_score(actual, pred)
    print("Recall Score: ", recall)
    mcc = matthews_corrcoef(actual, pred)
    print("MCC: ", mcc)
    roc = roc_auc_score(actual, pred, multi_class='ovr')
    print("ROC: ", roc)
    kappa = cohen_kappa_score(actual, pred)
    print("Kappa: ", kappa)
    mae = mean_absolute_error(actual, pred)
    print("Mean absolute error: ", mae)
    rmse = mean_squared_error(actual, pred, squared=False)
    print("Root Mean Squared Error: ", rmse)
    rae = sum(np.absolute(np.array(pred) - np.array(actual))) / sum(np.absolute(np.array(actual) - np.mean(np.array(y_train))))
    print("Relative Absolute Error: ", rae)
    rrse = math.sqrt(sum((np.array(actual) - np.array(pred))**2) / sum((np.array(actual) - np.mean(np.array(actual)))**2))
    print("Root Relative Squared Error: ", rrse)
    print("\n")
    print("="*60)
    print("\n")
    num += 1
    results = results.append({'S.no': num, 'Model': model, 'Execution Time': t, 
                              'Accuracy': accuracy, 'F1-score': fscore, 'Precision': precision, 
                              'Recall': recall, 'MCC': mcc, 'ROC': roc, 'Kappa': kappa, 
                              'Mean Absolute Error': mae, 'Root Mean Squared Error': rmse, 
                              'Root Absolute Error': rae, 'Root Relative Squared Error': rrse}, ignore_index=True)

In [9]:
def train_and_predict(model, clf):
    print('\033[1m' + model + '\033[0m')
    start = time.process_time()
    clf.fit(x_train, y_train)
    t = time.process_time() - start
    print("Execution time: ", t)
    predicted = clf.predict(x_test)
    get_report(y_test, predicted, model, t)

In [10]:
models = [ 
    ('Naive Bayes', BernoulliNB()),
    ('KNN', KNeighborsClassifier(n_neighbors=9, weights = "distance")),
    ('Decision Tree', tree.DecisionTreeClassifier(random_state = 0, max_depth = 8, criterion = "entropy")),
    ('Logistic Regression', LogisticRegression(C = 0.001, max_iter=1000)),
    ('Random Forest', RandomForestClassifier(n_jobs = -1, n_estimators = 500, warm_start = True, max_depth = 6, min_samples_leaf = 2, max_features = 'sqrt', verbose = 0))
]

In [11]:
for model, clf in models:
    train_and_predict(model, clf)
# 0.7675996983542563
# 0.77265670052788
# 0.8788981058421683
# 0.7045646098567183
# 0.8106729361664374

Naive Bayes
Execution time:  0.23479256300000007
Confusion matrix: 
 [[9431  280]
 [4959 7873]]
Accuracy:  0.7675996983542563
F1 Score:  0.7503454848701453
Precision Score:  0.9656568134429044
Recall Score:  0.6135442643391521
MCC:  0.6026029946361382
ROC:  0.792355491246911
Kappa:  0.5523463977504013
Mean absolute error:  0.23240030164574368
Root Mean Squared Error:  0.4820791445870104
Relative Absolute Error:  0.46039247609606737
Root Relative Squared Error:  0.9735335038232122




KNN
Execution time:  0.032019658000000284
Confusion matrix: 
 [[9412  299]
 [4624 8208]]
Accuracy:  0.7816173535021959
F1 Score:  0.7692956558414172
Precision Score:  0.9648524744328201
Recall Score:  0.6396508728179551
MCC:  0.6219957917808627
ROC:  0.804430523423703
Kappa:  0.5775784662011716
Mean absolute error:  0.2183826464978042
Root Mean Squared Error:  0.46731429091972376
Relative Absolute Error:  0.4326230501662416
Root Relative Squared Error:  0.943716657594644




Decision Tree
Execution time:  

In [12]:
for i in range(3):
    for j in range(i+1, 4):
        for k in range(j+1, 5):
            softVotingClf = VotingClassifier([models[i], models[j], models[k]], voting='soft')
            train_and_predict("Soft Voting Ensemble learning with models: " + models[i][0] + ", " + models[j][0] + ", " + models[k][0], softVotingClf)

Soft Voting Ensemble learning with models: Naive Bayes, KNN, Decision Tree
Execution time:  1.5202084539999987
Confusion matrix: 
 [[9442  269]
 [3865 8967]]
Accuracy:  0.8166171317038549
F1 Score:  0.8126699293094073
Precision Score:  0.9708748375920312
Recall Score:  0.6987998753117207
MCC:  0.675746674169677
ROC:  0.8355496647694427
Kappa:  0.642180401005628
Mean absolute error:  0.18338286829614514
Root Mean Squared Error:  0.42823225975648443
Relative Absolute Error:  0.3632873632718348
Root Relative Squared Error:  0.8647925490492075




Soft Voting Ensemble learning with models: Naive Bayes, KNN, Logistic Regression
Execution time:  17.970856244000004
Confusion matrix: 
 [[9428  283]
 [5646 7186]]
Accuracy:  0.7369915273033758
F1 Score:  0.7079454214078126
Precision Score:  0.96211005489356
Recall Score:  0.5600062344139651
MCC:  0.5584920077067291
ROC:  0.7654320122744319
Kappa:  0.49745476430274127
Mean absolute error:  0.26300847269662425
Root Mean Squared Error:  0.512843516

In [13]:
for i in range(3):
    for j in range(i+1, 4):
        for k in range(j+1, 5):
            softVotingClf = VotingClassifier([models[i], models[j], models[k]], voting='hard')
            train_and_predict("Hard Voting Ensemble learning with models: " + models[i][0] + ", " + models[j][0] + ", " + models[k][0], softVotingClf)

Hard Voting Ensemble learning with models: Naive Bayes, KNN, Decision Tree
Execution time:  1.691632348999974
Confusion matrix: 
 [[9441  270]
 [3829 9003]]
Accuracy:  0.8181697200904937
F1 Score:  0.8145668400814295
Precision Score:  0.970883209317373
Recall Score:  0.70160536159601
MCC:  0.6780562972357416
ROC:  0.8369009199082922
Kappa:  0.6450524592863778
Mean absolute error:  0.1818302799095063
Root Mean Squared Error:  0.4264156187447949
Relative Absolute Error:  0.36021163571631615
Root Relative Squared Error:  0.8611239379732936




Hard Voting Ensemble learning with models: Naive Bayes, KNN, Logistic Regression
Execution time:  37.207541311
Confusion matrix: 
 [[9422  289]
 [5723 7109]]
Accuracy:  0.7333096748436322
F1 Score:  0.7028175976272862
Precision Score:  0.9609353879426872
Recall Score:  0.5540056109725686
MCC:  0.5528688212452704
ROC:  0.7621227725339622
Kappa:  0.49084385367342065
Mean absolute error:  0.2666903251563678
Root Mean Squared Error:  0.5164206862204184


In [14]:
for i in range(3):
    for j in range(i+1, 4):
        for k in range(j+1, 5):
            softVotingClf = StackingClassifier([models[i], models[j], models[k]], final_estimator=LogisticRegression())
            train_and_predict("Stacking Ensemble learning with models: " + models[i][0] + ", " + models[j][0] + ", " + models[k][0], softVotingClf)

Stacking Ensemble learning with models: Naive Bayes, KNN, Decision Tree
Execution time:  109.13571057800004
Confusion matrix: 
 [[9448  263]
 [3849 8983]]
Accuracy:  0.8175930444040279
F1 Score:  0.8137512455838392
Precision Score:  0.9715552671425481
Recall Score:  0.7000467581047382
MCC:  0.6775124030232775
ROC:  0.836482034185723
Kappa:  0.6440419620973461
Mean absolute error:  0.18240695559597214
Root Mean Squared Error:  0.42709127314424505
Relative Absolute Error:  0.3613540488083659
Root Relative Squared Error:  0.8624883865337759




Stacking Ensemble learning with models: Naive Bayes, KNN, Logistic Regression
Execution time:  272.160382839
Confusion matrix: 
 [[9417  294]
 [4720 8112]]
Accuracy:  0.7775806236969347
F1 Score:  0.7639137395234957
Precision Score:  0.9650249821556032
Recall Score:  0.6321695760598504
MCC:  0.6163484376200582
ROC:  0.8009473150611269
Kappa:  0.5702857301444495
Mean absolute error:  0.22241937630306524
Root Mean Squared Error:  0.47161358791182556


In [15]:
results.to_csv('results2.csv', index=False)